# 고객 데이터 가공

#### 고객 데이터는 현재 존재하지 않음 -> 인공지능 학습을 위한 가상의 고객 데이터 생성

### 필수 패키지 정리

In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import random
import pymysql

### 데이터 읽어오기

In [ ]:
#소비데이터
file_path = "/Users/isanghyeon/Downloads/소비데이터.csv"
#데이터 읽기
df = pd.read_csv(file_path, encoding='cp949')

### 고객 데이터 생성을 위한 세부 값 설정

In [ ]:
#샘플 고객데이터 수량 설정
n_customers = 20000
#성별 설정 SQL문으로 진행하게 될때는 유저데이터에서 JOIN하여 값 얻기
gender = ["남성", "여성"]
#pattern_name 설정
pattern_name = ["첫번째","두번째","세번째","2025년","2024년","6월","5월","1월","2월","3월","4월","마지막"]
#소비패턴 값 저장 리스트
pattern_rows = []
pattern_detail_rows = []

### 소비데이터명과 대카테고리명 mapping
#### 성별별 금액에 관해 가중치 부여 목적

In [1]:
category_csi_map = {
    '모든가맹점': ['소비지출전망CSI'],
    '모빌리티': ['교통비 및 통신비 지출전망CSI'],
    '대중교통': ['교통비 및 통신비 지출전망CSI'],
    '통신': ['교통비 및 통신비 지출전망CSI'],
    '생활': ['소비지출전망CSI', '주거비 지출전망CSI'],
    '쇼핑': ['내구재 지출전망CSI', '의류비 지출전망CSI'],
    '외식/카페': ['외식비 지출전망CSI'],
    '뷰티/피트니스': ['의류비 지출전망CSI'],
    '금융/포인트': ['가계저축전망CSI'],
    '병원/약국': ['의료·보건비 지출전망CSI'],
    '문화/취미': ['교양·오락·문화생활비 지출전망CSI'],
    '숙박/항공': ['여행비 지출전망CSI']
}

### 성별에 관해서 금액 가중치 비중 저장

In [ ]:
#남성가중치
man_dic = {}
#남성데이터의 소비데이터명과 소비데이터값 평균 매칭
for i in range(len(df)):
    if df["CSI분류코드별"].iloc[i] == "남자":
        key = df["CSI코드별"].iloc[i]
        value = df["평균"].iloc[i]
        man_dic[key] = value
#카테고리명과 소비데이터값 평균 매칭
man_category_weights = {}
for category, csi_keys in category_csi_map.items():
    values = [man_dic.get(k, 100) for k in csi_keys]
    man_category_weights[category] = sum(values) / len(values)
#정규화진행
total = sum(man_category_weights.values())
man_normalized_weights = {k: round(v / total, 4) for k, v in man_category_weights.items()}
#남성 가중치값 저장
man_weight = []
for k, v in man_normalized_weights.items():
    man_weight.append(v)


#여성가중치
woman_dic = {}
#여성데이터의 소비데이터명과 소비데이터값 평균 매칭
for i in range(len(df)):
    if df["CSI분류코드별"].iloc[i] == "여자":
        key = df["CSI코드별"].iloc[i]
        value = df["평균"].iloc[i]
        woman_dic[key] = value
#카테고리명과 소비데이터값 평균 매칭
woman_category_weights = {}
for category, csi_keys in category_csi_map.items():
    values = [woman_dic.get(k, 100) for k in csi_keys]
    woman_category_weights[category] = sum(values) / len(values)
#정규화진행
total = sum(woman_category_weights.values())
woman_normalized_weights = {k: round(v / total, 4) for k, v in woman_category_weights.items()}
#여성 가중치값 저장
woman_weight = []
for k, v in woman_normalized_weights.items():
    woman_weight.append(v)

### 카테고리 설정 및 번호 부여

In [ ]:
#카테고리명
categories = ['모든가맹점','모빌리티','대중교통','통신','생활','쇼핑','외식/카페','뷰티/피트니스','금융/포인트','병원/약국','문화/취미','숙박/항공']
#카테고리명 랜덤 번호 부여
category_id_map = {cat: i+1 for i, cat in enumerate(categories)}

### 날짜 변환 함수 준비

In [ ]:
def random_datetime_within_days(days):
    now = datetime.now()
    random_days = random.randint(0, days)
    random_seconds = random.randint(0, 86400)  # 하루 중 랜덤 시간
    return (now - timedelta(days=random_days, seconds=random_seconds)).strftime("%Y-%m-%d %H:%M:%S")

### 데이터 가공 시작

In [ ]:
# 위에서 설정한 유저 수만큼 데이터 만들기 시작
for i in range(n_customers):
    user_id = i + 1
    # 남/여 성별에 관해서 랜덤 선택
    gender_choice = random.choice(gender)
    pattern_id = i + 1
    # 365일 이내로 랜덤 데이터 생성
    created_at = random_datetime_within_days(365)
    # 위에서 설정한 값을 통해 pattern_rows 리스트 저장 시작
    pattern_rows.append({
        "pattern_id": pattern_id,
        "user_id": random.randint(2,5002),
        "pattern_name": random.choice(pattern_name),
        "created_at": created_at,
        "gender_choice": gender_choice
    })
    # 금액 가중치 비율 설정
    weights = woman_weight if gender_choice == "여성" else man_weight

    # 2~6개의 소비패턴 선택을 위한 과정
    k = random.randint(2, 6)
    # 카테고리 목록 중 선택
    selected_cats = random.sample(categories, k=k)
    for cat in selected_cats:
        idx = categories.index(cat)
        weight = weights[idx]
        # 카테고리에 랜덤 소비금액 설정 10,000 ~ 500,000원 랜덤 설정
        base_amount = random.randint(10_000, 500_000)
        #가중치 부여
        amount = int(base_amount * (1 + weight))
        if amount > 0:
            # 0 이상인 값만 저장
            pattern_detail_rows.append({
                "pattern_id": pattern_id,
                "benefitcategory_id": category_id_map[cat],
                "amount": round(amount, -3)
            })
    #소비패턴 소비값 여성, 남성별 가중치 부여하여 데이터 생성
    weights = woman_weight if gender_choice == "여성" else man_weight


    #추가적으로 없은 소비패턴 추가 부여
    for cat, weight in zip(categories, weights):
        if random.random() < 0.25:
            #소비값 생성과 가중치 부여
            base_amount = random.randint(10_000, 500_000)
            amount = int(base_amount * (1+weight))
            if amount > 0:
                pattern_detail_rows.append({
                    "pattern_id": pattern_id,
                    "benefitcategory_id": category_id_map[cat],
                    "amount": round(amount,-3)
                })

### 저장된 값 dataframe 값으로 저장

In [ ]:
df_pattern_all = pd.DataFrame(pattern_rows)
df_pattern = df_pattern_all[["pattern_id", "user_id", "pattern_name", "created_at"]]
df_detail = pd.DataFrame(pattern_detail_rows)
df_detail = df_detail.drop_duplicates(subset=["pattern_id", "benefitcategory_id"], keep='first')

### database 저장
#### 위에서 가공된 데이터를 UserConsumptionPattern 테이블에 저장하기 위한 코드

In [ ]:
df_pattern = df_pattern.where(pd.notnull(df_pattern), None)
conn = pymysql.connect(
    host='localhost',
    user='cardgarden',
    password='1234',
    database='cardgarden',
    charset='utf8mb4'
)
cursor = conn.cursor()

insert_sql = """
INSERT INTO UserConsumptionPattern (user_id, pattern_name, created_at)
VALUES (%s, %s, %s)
"""

# pattern_id 실제값 저장용 리스트
actual_pattern_ids = []

try:
    for _, record in df_pattern.iterrows():
        cursor.execute(insert_sql, (
            record["user_id"],
            record["pattern_name"],
            record["created_at"]
        ))
        # LAST_INSERT_ID()를 통해 실제 pattern_id값을 가져옴
        pattern_id = cursor.lastrowid
        actual_pattern_ids.append(pattern_id)
    conn.commit()
except Exception as e:
    print("에러 발생:", e)
    conn.rollback()
finally:
    cursor.close()
    conn.close()

#### 위에서 가공된 데이터를 UserConsumptionPatternDetail 테이블에 저장하기 위한 코드

In [ ]:
# 먼저, 패턴id를 매핑할 딕셔너리 생성
old_to_new = {old: new for old, new in zip(df_pattern["pattern_id"], actual_pattern_ids)}

# df_detail의 pattern_id를 실제 pattern_id로 변환
df_detail["pattern_id"] = df_detail["pattern_id"].map(old_to_new)


df_detail = df_detail.where(pd.notnull(df_detail), None)
conn = pymysql.connect(
    host='localhost',
    user='cardgarden',
    password='1234',
    database='cardgarden',
    charset='utf8mb4'
)
cursor = conn.cursor()
insert_sql = """
INSERT INTO UserConsumptionPatternDetail (pattern_id, benefitcategory_id, amount)
VALUES (%s, %s, %s)
"""

try:
    for _, record in df_detail.iterrows():
        cursor.execute(insert_sql, (
            record["pattern_id"],
            record["benefitcategory_id"],
            record["amount"]
        ))
    conn.commit()
except Exception as e:
    print("에러 발생:", e)
    conn.rollback()
finally:
    cursor.close()
    conn.close()